</br>
<h1 align="center" style="color:green">Case-Based Hands-On EDA Activity</br> UK-based Online Gift Retailer Dataset</h1>
<div style="text-align:center">Prepared by <b>Kirk T. Braga, Jan Vincent G. Elleazar, Michael Suacito, and Carl Jacob Mateo</b><br>
CSELEC3C - Data Analysis and Visualization <br>
<b>UST College of Information and Computing Sciences</b>
</div>

***
## Scenario
You’ve been hired as a junior data-analytics duo at a UK-based online gift retailer. The CEO wants to understand what truly drives revenue and returns across two years of transactions to guide Q4 promotions and inventory buys. Your job is to clean, explore, and explain the dataset—surfacing patterns by product, country, customers, and time.

## Learning Outcomes
- Clean messy transaction data and build derived fields for retail EDA.
- Summarize and visualize customers, orders, products, countries, and time.
- Quantify returns and their impact on revenue and top products.
- Communicate insights with clear plots and a short slide story.
***

# Step-by-Step Tasks
## 1) Project setup
Create a folder: ```retail_eda_pair_Surname1_Surname2/``` <br>
Subfolders: ```notebooks/```, ```figures/```, ```reports/```, ```exports/```.
## 2) Load & union the data
- Load both sheets and concatenate into a single table.
- Parse ```InvoiceDate``` as datetime (it includes time of day).
- Add a column ```YearSheet``` showing the source sheet (2009–2010 vs 2010–2011) for comparisons.

In [1]:
import pandas as pd
import numpy as np

# Load csv dataset
data1 = pd.read_csv('/Users/jjelleazarr/Documents/GitHub/Data-Visualization-and-Analytics/retail_eda_pair_Braga_Elleazar_Mateo_Suacito/notebooks/online_retail_II.xlsx - Year 2009-2010.csv')
data2 = pd.read_csv('/Users/jjelleazarr/Documents/GitHub/Data-Visualization-and-Analytics/retail_eda_pair_Braga_Elleazar_Mateo_Suacito/notebooks/online_retail_II.xlsx - Year 2010-2011.csv')
# Note: Adjust the file paths on your system as needed

# Concatenate datasets
data = pd.concat([data1, data2], ignore_index=True)

# Parse InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Add a column 'YearSheet' showing the source sheet (2009-2010 vs 2010-2011)
data['YearSheet'] = np.where(data['InvoiceDate'].dt.year == 2009, '2009-2010', '2010-2011')

# Display the first few rows of the combined dataset
print(data.head())
print("\n")
# Display the last few rows of the combined dataset
print(data.tail())

/var/folders/vs/zcjd1wds7_78_tmdl741y03m0000gn/T/ipykernel_7720/3880971041.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])


  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  YearSheet  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  2009-2010  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009-2010  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009-2010  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  2009-2010  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  2009-2010  


        Invoice StockCode                      Description  Quantity  \
1067366  581587     22899     CHILDREN'S APRON DOLLY GIRL          6   
1067367

## 3) Basic cleaning
- Drop exact duplicate rows.
- Handle missing Customer ID (report % missing; keep for product/country EDA, but exclude from customer-level analyses).
- Remove or flag zero/negative Price rows (report counts).
- Create returns flag: Is_Return = Quantity < 0.
- Create Revenue: Revenue = Quantity * Price (negative revenue for returns).

In [2]:
# Drop exact duplicate rows
data = data.drop_duplicates()

# Report percentage of missing Customer ID
missing_customer_id_pct = data['Customer ID'].isna().mean() * 100
print(f"Percentage of missing Customer ID: {missing_customer_id_pct:.2f}%")

# Remove rows with missing Customer ID
data = data.dropna(subset=['Customer ID'])

# Check for missing customer IDs again
missing_customer_id_pct = data['Customer ID'].isna().mean() * 100
print(f"Percentage of missing Customer ID after removal: {missing_customer_id_pct:.2f}%")

# Flag zero/negative Price rows
zero_negative_price_count = (data['Price'] <= 0).sum()
print(f"Count of zero/negative Price rows: {zero_negative_price_count}")

# Remove rows with zero/negative Price
data = data[data['Price'] > 0]

# Check for zero/negative Price rows again
zero_negative_price_count = (data['Price'] <= 0).sum()
print(f"Count of zero/negative Price rows after removal: {zero_negative_price_count}")

# Create returns flag
data['Is_Return'] = data['Quantity'] < 0

# Show counts of returns vs non-returns
returns_counts = data['Is_Return'].value_counts()
print("Counts of returns vs non-returns:")
print(returns_counts)

# Create Revenue column (negative revenue for returns)
data['Revenue'] = data['Quantity'] * data['Price']

# Confirm no Price <= 0 rows
zero_negative_price_count = (data['Price'] <= 0).sum()
print(f"Final check - Count of zero/negative Price rows: {zero_negative_price_count}")

print('--------------------------------')

# Display the first few rows of the cleaned dataset
print(data.head())

# Display the last few rows of the cleaned dataset
print(data.tail())

Percentage of missing Customer ID: 22.76%
Percentage of missing Customer ID after removal: 0.00%
Count of zero/negative Price rows: 70
Count of zero/negative Price rows after removal: 0
Counts of returns vs non-returns:
Is_Return
False    779425
True      18390
Name: count, dtype: int64
Final check - Count of zero/negative Price rows: 0
--------------------------------
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  YearSheet  \
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  2009-2010   
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  2009

## 4) Time features
From ``InvoiceDate``, derive:
- ``Year, Quarter, Month, DayOfWeek, Hour``
- ``InvoiceDateFloorMonth`` (month start) for monthly rollups. 

Checks:
- Value counts for ``Hour`` and ``DayOfWeek`` to see busy times.

In [ ]:
# Insert code for time features here

## 5) Orders & customers
- Define order level by ``Invoice`` (each invoice is an order).
- Compute AOV (Average Order Value), items per order, and orders per customer (where ``Customer ID`` exists).
- Classify customers: New vs Repeat per sheet-year (first purchase month per customer).

Outputs:
- Table: ``n_orders, n_customers (non-null), AOV, items_per_order`` per ``YearSheet``.
- Bar chart: New vs Repeat revenue share per ``YearSheet``.

In [ ]:
# Insert code for orders & customers here

## 6) Product & country profiles
- Top 10 products by revenue (exclude returns; show ``StockCode, Description, Revenue, Quantity``).
- Top 10 countries by revenue and share of total (show UK vs non-UK split).
- Return-prone products: top items by return rate = (abs(negative Qty) / total Qty), min 200 units sold threshold.

Outputs:
- Two ranked tables (products & countries).
- One chart: UK vs Rest-of-World revenue share.

In [ ]:
# Insert code for product & country profiles here

## 7) Time-series EDA
- Monthly revenue line for the full period (net of returns). Annotate visible spikes/dips.
- Seasonality: average revenue by Month (1–12) over the two years.
- Intra-day pattern: average revenue by Hour (optional if time granularity is adequate).

Outputs:
- Line chart: monthly revenue (with returns accounted for).
- Bar/line: revenue by month number (seasonality).
- Line: revenue by hour.

In [ ]:
# Insert code for time-series EDA here

## 8) Returns analysis
- Overall return rate = (sum of negative Quantity) / (sum of abs(Quantity)).
- Return impact on revenue (sum of negative Revenue vs net revenue).
- Returns by Country and Top 10 products (from §6).

Outputs:
- Table: return rate & revenue impact (overall, by country).
- Short paragraph: business impact & hypotheses (e.g., product quality, shipping damages, wrong descriptions).

In [ ]:
# Insert code for returns analysis here

## 9) Customer-level snapshots (where ``Customer ID`` is present)
- RFM-style quick view (no need to segment):
- Recency (days since last purchase at dataset end), Frequency (# invoices), Monetary (total net revenue).
- Histograms for R, F, M with brief observations.

Outputs:
- Three small histograms + 3 bullets of commentary (e.g., “Most customers are low-frequency one-time buyers; long tail of high-value repeaters.”)

In [ ]:
# Insert code for customer-level snapshots here

## 10) Synthesis & recommendations
- 5–7 clear insights (each backed by one plot/table).
- 3 actions the retailer could test next month (e.g., “Increase stock for top product X in Country Y; reduce returns for StockCode Z via packaging or description clarity; run a weekend promo at the top revenue hour.”)